# Каменев Нил БКЛ211
## Проект

In [2]:
# Импортируем необходимые библиотеки
import requests
from bs4 import BeautifulSoup
import re
from pymorphy2 import MorphAnalyzer
import urllib.request
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ivigtel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# Собираем корпус текстов
url_list = {'http://az.lib.ru/g/gogolx_n_w/text_1832_oryvki_iz_neizv_dramy.shtml': 'Н.В. Гоголь. "Черновые наброски. Отрывки из неизвестной драмы" — lib.ru', 'http://az.lib.ru/g/gogolx_n_w/text_0280.shtml': 'Н.В. Гоголь. "Ночи на вилле" — lib.ru', 'http://az.lib.ru/g/gogolx_n_w/text_1838_nabroski.shtml': 'Н.В. Гоголь. "Черновые наброски. Наброски драмы из украинской истории" — lib.ru', 'http://az.lib.ru/g/gogolx_n_w/text_1835_alfred_var.shtml': 'Н.В. Гоголь. "Черновые наброски. Варианты" — lib.ru', 'http://az.lib.ru/g/gogolx_n_w/text_0270.shtml': 'Н.В. Гоголь. "Повесть о капитане Копейкине" — lib.ru', 'http://az.lib.ru/g/gogolx_n_w/text_0350.shtml': 'Н.В. Гоголь. "Мёртвые Души (Том второй)" — lib.ru'}

def get_corpus(url_list):
    corpus = []
    for key in url_list:
        # Загружаем страницу
        page = requests.get(key)
        soup = BeautifulSoup(page.content, 'html.parser')
        # Удаляем все теги и лишние спецсимволы
        text_f = re.search('/table>(.*)', str(soup), flags= re.U | re.DOTALL)
        cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        cleantext = re.sub(cleanr, '', text_f.group(1))
        cleaner = re.sub('\n', ' ', cleantext)
        cleaner_2 = re.sub('\r', ' ', cleaner)
        cleaner_3 = re.sub('\x00', ' ', cleaner_2)
        final_cleaner = re.sub('\xa0', ' ', cleaner_3)
        # Разбиваем текст на предложения
        sentences = nltk.sent_tokenize(final_cleaner)
        # Добавляем каждое предложение в корпус с указанием источника
        for s in sentences:
            if re.search(r'\w', s):
                corpus.append({'source': key, 'title': url_list[key], 'sentence': s})
    return corpus

In [13]:
# Морфологический анализ текстов
corpus = get_corpus(url_list)
def analyze_text(corpus):
    morph = MorphAnalyzer() #использование морфоанализатора pymorphy2
    analyzed_corpus = []
    for doc in corpus:
        analyzed_doc = []
        words = re.findall(r'\w+', doc['sentence'])
        for w in words:
            # Анализируем каждое слово и добавляем информацию о лемме и теге
            p = morph.parse(w)[0]
            analyzed_doc.append({'word': w, 'lemma': p.normal_form, 'tag': p.tag.POS})
        analyzed_corpus.append({'source': doc['source'], 'title': doc['title'], 'sentence': doc['sentence'], 'words': analyzed_doc})
    return analyzed_corpus
analyze_text(corpus)

[{'source': 'http://az.lib.ru/g/gogolx_n_w/text_1832_oryvki_iz_neizv_dramy.shtml',
  'title': 'Н.В. Гоголь. "Черновые наброски. Отрывки из неизвестной драмы" — lib.ru',
  'sentence': '         <!--  <i linotypedate=25.12.2009 SMTPinIDX=.1.1.17.66.95.24.77.78  >  -->            Николай Васильевич Гоголь      Черновые наброски      ОТРЫВКИ ИЗ НЕИЗВЕСТНОЙ ДРАМЫ      I       Баскаков.',
  'words': [{'word': 'i', 'lemma': 'i', 'tag': None},
   {'word': 'linotypedate', 'lemma': 'linotypedate', 'tag': None},
   {'word': '25', 'lemma': '25', 'tag': None},
   {'word': '12', 'lemma': '12', 'tag': None},
   {'word': '2009', 'lemma': '2009', 'tag': None},
   {'word': 'SMTPinIDX', 'lemma': 'smtpinidx', 'tag': None},
   {'word': '1', 'lemma': '1', 'tag': None},
   {'word': '1', 'lemma': '1', 'tag': None},
   {'word': '17', 'lemma': '17', 'tag': None},
   {'word': '66', 'lemma': '66', 'tag': None},
   {'word': '95', 'lemma': '95', 'tag': None},
   {'word': '24', 'lemma': '24', 'tag': None},
   {'word

In [62]:
# Функция поиска
analyzed_corpus = analyze_text(corpus)
query = input("Введите список токенов через пробел: ") # запрос ввода списка токенов
def search(query, analyzed_corpus): # Поиск по корпусу
    query_tokens = query.split() # разделение строки на список токенов
    result = []
    lemmatized_tokens = []
    for token in query_tokens:
        if '+' in token:
            lemmo4ka, postegik = token.split('+')
            lemmatized_tokens.append(MorphAnalyzer().parse(lemmo4ka)[0].normal_form.lower()) # лемматизация токенов для запроса леммы с POS-тегом
        else:
            lemmatized_tokens.append(MorphAnalyzer().parse(token)[0].normal_form.lower()) # лемматизация токенов для запроса леммы
    for doc in analyzed_corpus:
        for i in range(len(doc['words']) - len(query_tokens) + 1):
            match = True
            for j in range(len(query_tokens)):
                # Проверяем, соответствует ли текущее слово запросу
                if '"' in query_tokens[j]: # словоформа в двойных кавычках (находит только заданную форму)
                    if not doc['words'][i+j]['word'].lower() == re.search(r'\w+', query_tokens[j].lower()).group(0):
                        match = False
                        break
                elif ('"' not in query_tokens[j]) and ('+' not in query_tokens[j]) and (not query_tokens[j].isupper()): # лемма (находит вхождения во всех формах)
                    if not doc['words'][i+j]['lemma'].lower() == lemmatized_tokens[j]:
                        match = False
                        break
                elif '+' in query_tokens[j]: # лемма и POS-тег (находит все формы, отмеченные данным тегом)
                    lemmochka, posik = query_tokens[j].split('+')
                    if not (doc['words'][i+j]['lemma'].lower() == lemmatized_tokens[j]) or not (doc['words'][i+j]['tag'] == posik):
                        match = False
                        break
                else: # только POS-тег(и)
                    if not doc['words'][i+j]['tag'] == query_tokens[j]:
                        match = False
                        break
            if match:
                result.append({'source': doc['source'], 'title': doc['title'], 'sentence': doc['sentence']}) # вывод предложений с мета-информацией
                break
    return result
search(query, analyzed_corpus)

Введите список токенов через пробел: сказали+INFN "жизнью" жертвовали


[{'source': 'http://az.lib.ru/g/gogolx_n_w/text_0270.shtml',
  'title': 'Н.В. Гоголь. "Повесть о капитане Копейкине" — lib.ru',
  'sentence': '"Пойду в комиссию,  -  говорит  Копейкин,  скажу:  так и так, проливал, в некотором роде, кровь,  относительно  сказать, жизнью жертвовал".'}]